In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as rmse

import warnings
warnings.filterwarnings('ignore')

: 

##Loading the data

In [ ]:
aq = pd.read_csv('/content/drive/MyDrive/hw2_dataset/AirQualityUCI.csv', sep=';', decimal=',', header=0)

: 

In [ ]:
aq.head(10)

: 

We have trash columns 'Unnamed:15' and 'Unnamed:16', so we should drop them.

In [ ]:
aq.drop(['Unnamed: 15','Unnamed: 16'], axis=1, inplace=True)
aq.head(10)

: 

In [ ]:
aq.shape

: 

In [ ]:
aq.info()

: 

In [ ]:
aq.describe()

: 

##Handling Date and Time columns

In [ ]:
aq['Date'] = pd.to_datetime(aq['Date'],dayfirst=True)

aq['Time'] = pd.to_datetime(aq['Time'],format= '%H.%M.%S' ).dt.time

aq.head()

: 

In [ ]:
aq.dtypes

: 

##Removing Null/Missing Valus

Missing values are labeled with '-200' so I'll handle those instances.

In [ ]:
aq.replace(to_replace=-200, value=np.nan, inplace=True)

aq.describe()

: 

In [ ]:
aq.isna().sum()

: 

In [ ]:
print('Percentage of missing values in each column of this dataset')
for col in aq.columns:
  print(f'{col}:{np.round((aq[col].isna().sum()/aq.shape[0]) * 100, 2)}%')

: 

We see that column 'NMHC(GT)' has around 90% of null values. This means that, if we were to drop these rows we would lose 90% of our dataset. Furthermore, we still have a column with information about Benzene (C6H6) concentrations (benzene is a hydrocarbon) through the columns C6H6(GT) and PT08.S2(NMHC), so we haven't completely lost the entirety of the hydrocarbon readings. Instead, I'll opt for dropping this feature from the dataset. For all other columns that contain na values, we'll replace them with mean values of each column, to not lose any data. For 'Date' and 'Time' columns, we will remove rows that contain null values.

In [ ]:
aq.drop('NMHC(GT)', axis=1, inplace=True)

cols = ['CO(GT)', 'PT08.S1(CO)', 'C6H6(GT)','PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)','PT08.S5(O3)', 'T', 'RH', 'AH']

for col in cols:
    aq[col] = aq[col].fillna(aq[col].mean())

aq.dropna(inplace=True)

aq.head(10)

: 

In [ ]:
aq.isna().sum()

: 

##Handling outliers

In [ ]:
num_cols = ['CO(GT)', 'PT08.S1(CO)', 'PT08.S2(NMHC)', 'C6H6(GT)',
       'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)',
       'T', 'RH', 'AH']

plt.figure(figsize=(12, 8))

for i, col in enumerate(num_cols, 1):
    plt.subplot(3, 4, i)
    plt.hist(aq[col], bins=20, color='skyblue', edgecolor='black')
    plt.title(col)
    plt.xlabel('Values')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

: 

From this histograms we can see that target variable C6H6(GT) is positively skewed, that means that its distribution is skewed towards higher values. Skewness can impact the performance of my regression model. I decided to handle outliers, as a way to deal with this problem (explained later).

In [ ]:
plt.figure(figsize=(8, 6))

# plotting box and whiskers
custom_colors = ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99']
sns.set_palette(sns.color_palette(custom_colors))

sns.boxplot(data=aq)
plt.xticks(rotation='vertical')
plt.title('Air Quality Data')
plt.xlabel('Features')
plt.ylabel('Values')
plt.grid(True)
plt.tight_layout()
plt.show()

: 

As we can see, there are a lot of outliers in our dataset. I've set a threshold for outliers, which is determined by their distance from Q3 based on the interquartile range (IQR).

In [ ]:
Q1 = aq[num_cols].quantile(0.25)
Q3 = aq[num_cols].quantile(0.75)

IQR = Q3 - Q1

# making a mask that will filter out outliers in my dataset
mask = (aq[num_cols] < (Q1 - 1.5 * IQR)) | (aq[num_cols] > (Q3 + 1 * IQR))

mask.sum()

: 

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(aq['C6H6(GT)'], bins=30, color='skyblue', edgecolor='black')
plt.title('Distribution of C6H6(GT)')
plt.xlabel('C6H6(GT)')
plt.ylabel('Frequency')
plt.axvline(x=(Q3['C6H6(GT)'] + 1 * IQR['C6H6(GT)']), color='red', linestyle='--', label='Upper Threshold')
plt.axvline(x=(Q1['C6H6(GT)'] - 1.5 * IQR['C6H6(GT)']), color='orange', linestyle='--', label='Lower Threshold')
plt.legend()
plt.show()

: 

Considering that trimming or removing outliers would lead to data loss, I opted to replace outliers with maximum or minimum values falling within the non-outlier range. To set the maximum threshold, I utilized values within the IQR distance from Q3. This approach proved beneficial after tuning, demonstrating improved model performance.

In [ ]:
# identify outliers using the mask
mask_min = (aq[num_cols] < (Q1 - 1.5 * IQR))
mask_max = (aq[num_cols] > (Q3 + 1 * IQR))

min_non_outliers = aq[num_cols].apply(lambda x: x[~mask_min[x.name]].min(), axis=0)
max_non_outliers = aq[num_cols].apply(lambda x: x[~mask_max[x.name]].max(), axis=0)

# Replace outliers in selected columns
aq[num_cols] = np.where(mask_min, min_non_outliers, aq[num_cols])
aq[num_cols] = np.where(mask_max, max_non_outliers, aq[num_cols])

((aq[num_cols] < (Q1 - 1.5 * IQR)) | (aq[num_cols] > (Q3 + 1 * IQR))).sum()

: 

##Minor adjustments

Adding week day column.

In [ ]:
aq['Week Day'] = aq['Date'].dt.day_name()

# rearranging columns

cols = aq.columns.tolist()
cols = cols[:1] + cols[-1:] + cols[1:11]
aq_filt = aq[cols]
aq_filt.head(10)

: 

In [ ]:
aq_wed = aq_filt[aq_filt['Week Day'] == 'Monday']

#Plotting the mean hourly value of CO on Mondays

plt.figure(figsize=(10, 6))

sns.barplot(x='Time', y='PT08.S1(CO)', data=aq_wed.sort_values('Time'), palette='viridis')
plt.title('Mean Hourly Values of CO on Mondays', fontsize=16)
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('Mean CO Value', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

: 

We can see that CO emissions are at their peak between 7 and 9 o'clock and between 18 and 20 o'clock, beginnings and endings of office hours.

##Correlation

In [ ]:
plt.figure(figsize=(10, 8))

sns.set_style("whitegrid")

heatmap = sns.heatmap(aq.corr(), annot=True, cmap='YlGnBu', fmt=".2f")
plt.title('Correlation Matrix', fontsize=16)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()

plt.show()

: 

In [ ]:
plt.figure(figsize=(12, 10))


sns.set(style="ticks", palette="muted")

pair_plot = sns.pairplot(aq)
plt.tight_layout()
plt.show()

: 

Column such as T, RH, AH do not have strong correlation with other features. Eliminating the NMHC for being a redundant sensor (the C6H6 molecule is a Non-Methanic Hydrocarbon (NMHC)), so the correlation between those two sensors is exact (multicollinearity issue).

In [ ]:
aq_filt.drop('PT08.S2(NMHC)', axis=1, inplace=True)

: 

##Regression

In [ ]:
X = aq_filt[['CO(GT)', 'PT08.S1(CO)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)','PT08.S5(O3)']]
y = aq_filt['C6H6(GT)']

# scaling
scaler = RobustScaler()
scaled_features = scaler.fit_transform(X)

# using polynomial features
poly = PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(scaled_features)

X_train, X_test, y_train, y_test = train_test_split(X_poly,y, test_size = 0.3, random_state=1)
# using linear regression
model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("R2_score of Linear Regression:",r2(y_test,y_pred))
print("Mean Average Error of Linear Regression:",mae(y_test,y_pred))
print("Mean Squared Error of Linear Regression:",rmse(y_test,y_pred))

: 

An R2 score of 0.91 indicates that approximately 91% of the variance in the target variable ('C6H6(GT)') is explained by the features included in my model. On average, my model's predictions have an error of approximately 1.40 units (in the same units as my target variable).

In [ ]:
# plotting predicted and actual values to compare results
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.title('Predicted vs. Actual Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

: 

In [ ]:
y_pred = model.predict(X_test)
residuals = y_test - y_pred

sns.scatterplot(x=y_pred, y=residuals)
plt.title('Residual Plot')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='-')
plt.show()

: 

##Hyperparameter Tuning for Polynomial Regression degree

In [ ]:
param_grid = {'polynomialfeatures__degree': [2, 3, 4]}

# created a pipeline with PolynomialFeatures and LinearRegression
pipeline = make_pipeline(PolynomialFeatures(), LinearRegression())

grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X, y)
print("Best degree parameter:", grid_search.best_params_)

# re-run my model with best parameters
best_degree = grid_search.best_params_['polynomialfeatures__degree']
best_pipeline = make_pipeline(PolynomialFeatures(degree=best_degree), LinearRegression())
best_pipeline.fit(X, y)

# model's performance
y_pred = best_pipeline.predict(X)
print("R2 score of Polynomial Regression:", r2(y, y_pred))
print("Mean Absolute Error of Polynomial Regression:", mae(y, y_pred))
print("Root Mean Squared Error of Polynomial Regression:", rmse(y, y_pred))

: 

In [ ]:
# plotting predicted and actual values to compare results
plt.figure(figsize=(8, 6))
plt.scatter(y, y_pred, alpha=0.5)
plt.title('Predicted vs. Actual Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

: 

In [ ]:
y_pred = best_pipeline.predict(X)

# calculating residuals
residuals = y - y_pred

plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot for Polynomial Regression')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

: 

##Conclusion

The slight difference between the two models indicates that the data might not benefit significantly from polynomial transformations. The small improvement suggests that the default hyperparameters used in the initial model were already close to optimal. It's also possible that other hyperparameters beyond the degree of the polynomial could be fine-tuned for a better performance boost.